Full-Text Search searches the documents that match with a query and, optionally, sort the records according to their relevance to the query.

### Code


We have already created a table with embeddings. Lets do a basic keyword search

In [2]:
from dotenv import load_dotenv
import os,json 
from openai import AzureOpenAI
import psycopg2,os
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
import numpy as np

load_dotenv()


## Intializing connection and Creating a Table


 
DBUSER = os.environ["DBUSER"]
DBPASS = os.environ["DBPASS"]
DBHOST = os.environ["DBHOST"]
DBNAME = os.environ["DBNAME"]
# Use SSL if not connecting to localhost
DBSSL = "disable"
if DBHOST != "localhost":
    DBSSL = "require"
 
 
## initiate a connection
 
def initiate_connection():
    conn = psycopg2.connect(database=DBNAME, user=DBUSER, password=DBPASS, host=DBHOST, sslmode=DBSSL,port=38530)
    conn.autocommit = True
    return conn



###   Create table
 
## initialize a connection
conn = initiate_connection()
## create a table
cur = conn.cursor()


In [ ]:
table_name = "document_text_values"
cur.execute(f"create table {table_name}(file_id int, content text, description text)")




###   inject data
data = [
(1, 'The stock exchange predicts a bull market which would make many investors happy, but analysts warn of possibility of too much optimism and that in fact we are awaiting a bear market. As always predicting the future market is an uncertain game and all investors should follow their instincts and best practices.','Impact of bull market on investors'),
(2, 'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," the vast majority of investors are "bullish." Over lengthy time periods, the stock market has generally produced positive returns.','Market: positive and  negative sentiments'),
(3, 'During a bear market, market sentiment is negative, and investors begin to shift their money away from stocks and into fixed-income instruments as they wait for the stock market to recover. To summarise, the drop in stock market values has shaken investor confidence. As a result, investors keep their money out of the market, causing a general price decrease as outflow grows.','Bull and bear market'),
(4, ' the decrease in stock market values has shaken investor confidence, As they would have predicted that the market will fall more.  This causes investors to keep their money out of the market, causing a general price decrease as outflow grows. In bear market, value of stocks falls while in bull market the value of stocks falls. However the market has been producing the positive result over length periods.','Impact of price decrease in market')
]

## initialize a connection
conn = initiate_connection()
cur = conn.cursor()
execute_values(cur,f"INSERT INTO {table_name} (file_id,content,description) VALUES %s",data)



## print total number of records in the table
cur.execute(f"SELECT COUNT(*) as cnt FROM {table_name}")
num_records = cur.fetchone()[0]
print("Total number of records inserted: ",num_records)

Lets see a basic text search

In [3]:
## keyword search
def keyword_search(conn, query):
    cur = conn.cursor()
    cur.execute("SELECT file_id, content FROM document_text_values, plainto_tsquery('english', %s) query \
                 WHERE to_tsvector('english', content) @@ query ORDER BY ts_rank_cd(to_tsvector('english', content),\
                 query) DESC LIMIT 2", (query,))
    return cur.fetchall()

cur = conn.cursor()

# query = "Barcelona"

query = "market sentiment"
## print total number of records in the table
result = keyword_search(conn, query)
print(result)

# print(result.fetchall())


[(3, 'During a bear market, market sentiment is negative, and investors begin to shift their money away from stocks and into fixed-income instruments as they wait for the stock market to recover. To summarise, the drop in stock market values has shaken investor confidence. As a result, investors keep their money out of the market, causing a general price decrease as outflow grows.')]


Now we will get to know how this actually works by getting in to text search feature by postgres.

### Some Import Keyword Searching Functions in Postgres

We can say there are mainly 3 concepts in text search using Postgres

-  Preprocess the documents : ts_vector()
-  Query the processed document: ts_query()
-  Ranking the search results: ts_rank() and ts_rank_cd()


### Text Parsing - TsVector and TsQuery - Preprocessing and Querying

**tsvector**

- The tsvector is a sorted collection of key-value pair, where key represent lexeme and value represents lexeme's position.Lexemes are normalized representation of words. There are no stop words like 'on','at' etc in lexemes.

- to_tsvector() is Postgres’s internal function that returns a tsvector for the text or document passed as an argument, which can be used to perform efficient text searches. 

*Note: Lexeme is nothing but root form of a word.For eg: walk has "walks"," "walked," and "walking".*


ref: https://medium.com/geekculture/comprehend-tsvector-and-tsquery-in-postgres-for-full-text-search-1fd4323409fc

In [36]:
sql = "SELECT to_tsvector('Apple trees are beautiful. I love having apple each day along with grapes')"

def execute_statement(sql):
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()

execute_statement(sql)

[("'along':11 'appl':1,8 'beauti':4 'day':10 'grape':13 'love':6 'tree':2",)]

**tsquery**

* Each word’s normalized lexeme representation differs slightly from the word itself.For example word 'apple' transformed in to 'appl' lexeme. Searching 'apple' without using the to_tsquery() function returns false because Postgres internally casts the word 'apple' into the tsquery data type 'appl', which is not present in the ts vector.

* A tsquery contains search terms, which must be already-normalized lexeme

In [39]:
sql = "select to_tsquery('apple')"
## here :: is the cast operator
execute_statement(sql)

[("'appl'",)]

In [72]:

sql1 = "select to_tsvector('apple') @@ to_tsquery('apple')"
status = execute_statement(sql1)
print(status)

[(True,)]


In [49]:
sql = "select to_tsquery('runs'),to_tsquery('running'),to_tsquery('run')"
## here :: is the cast operator
execute_statement(sql)

[("'run'", "'run'", "'run'")]

**Types of ts_query**

Postgres provides 3 typs of ts_query convertion:

* plain_to_tsquery()
* websearch_to_tsquery()
* phrase_to_tsquery()

**plainto_tsquery:**

It transforms the unformatted text querytext to a tsquery value. The text is parsed and normalized much as for to_tsvector, then the & (AND) tsquery operator is inserted between surviving words

In [80]:
sql = "select plainto_tsquery('apple is healthy fruit')"

execute_statement(sql)

[("'appl' & 'healthi' & 'fruit'",)]

In [79]:
sql1 = "select to_tsvector('apple fruit is red in color') @@ plainto_tsquery('apple is healthy fruit')"
status = execute_statement(sql1)
print(status)

sql1 = "select to_tsvector('Eating apple fruit daily will give healthy benifits') @@ plainto_tsquery('apple is healthy fruit')"
status = execute_statement(sql1)
print(status)

[(False,)]
[(True,)]


We can see the second string matches because we have all norm words 'appl','health' and 'fruit'(& operator) while in first example we have dont have 'healthi'

**phraseto_tsquery:**

phraseto_tsquery behaves much like plainto_tsquery, except that it inserts the <-> (FOLLOWED BY) operator between surviving words instead of the & (AND) operator.

In [81]:
sql = "select phraseto_tsquery('apple is healthy fruit')"

execute_statement(sql)


[("'appl' <2> 'healthi' <-> 'fruit'",)]

In [88]:
sql1 = "select to_tsvector('apple fruit is red in color') @@ phraseto_tsquery('apple is healthy fruit')"
status = execute_statement(sql1)
print(status)

sql3 = "select to_tsvector('Eating apple fruit daily will give healthy benifits') @@ phraseto_tsquery('apple is healthy fruit')"
status = execute_statement(sql3)
print(status)


sql3 = "select to_tsvector('apple is healthy fruit in our country') @@ phraseto_tsquery('apple is healthy fruit')"
status = execute_statement(sql3)
print(status)

[(False,)]
[(False,)]
[(True,)]


We can see that the search result matches only if the words in our query is in same order.

**websearchto_tsquery**

Its is a combination of  plainto_tsquery() and phraseto_tsquery(). 
it also intakes operators given in search text unlike pther two and doesnot throw syndax errors.

In [97]:

sql = """select websearch_to_tsquery('"apple is healthy fruit"')"""
## here :: is the cast operator
execute_statement(sql)

[("'appl' <2> 'healthi' <-> 'fruit'",)]

In [98]:
sql = "select websearch_to_tsquery('apple is healthy fruit')"
## here :: is the cast operator
execute_statement(sql)

[("'appl' & 'healthi' & 'fruit'",)]

In [104]:
sql = """select websearch_to_tsquery('"fruit" or "vegetable"')"""
## here :: is the cast operator
execute_statement(sql)

[("'fruit' | 'veget'",)]




Lets see how to fetch from database

In [20]:
sql = "select file_id, content from document_text_values where to_tsvector(content) @@ to_tsquery('confidence');"

execute_statement(sql)

[(3,
  'During a bear market, market sentiment is negative, and investors begin to shift their money away from stocks and into fixed-income instruments as they wait for the stock market to recover. To summarise, the drop in stock market values has shaken investor confidence. As a result, investors keep their money out of the market, causing a general price decrease as outflow grows.'),
 (4,
  ' the decrease in stock market values has shaken investor confidence, As they would have predicted that the market will fall more.  This causes investors to keep their money out of the market, causing a general price decrease as outflow grows. In bear market, value of stocks falls while in bull market the value of stocks falls. However the market has been producing the positive result over length periods.')]



Note: 
- Here @@ is the match operator
- We can use &(and operator),|(or operator),!(not operator with ts_vector)

In [25]:
## example
sql_1 = "select file_id, content from document_text_values where to_tsvector(content) @@ to_tsquery('confidence & prediction & investors');"

print(execute_statement(sql_1))


sql_2 = "select file_id, content from document_text_values where to_tsvector(content) @@ to_tsquery('negative | economy');"
print(execute_statement(sql_2))

sql_3 = "select file_id, content from document_text_values where to_tsvector(content) @@ to_tsquery('!bear');"
print(execute_statement(sql_3))


sql_4 = "select file_id, content from document_text_values where to_tsvector(content) @@ to_tsquery('(!bear & predict)| instruments');"
print(execute_statement(sql_4))


[(4, ' the decrease in stock market values has shaken investor confidence, As they would have predicted that the market will fall more.  This causes investors to keep their money out of the market, causing a general price decrease as outflow grows. In bear market, value of stocks falls while in bull market the value of stocks falls. However the market has been producing the positive result over length periods.')]
[(2, 'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," the vast majority of investors are "bullish." Over lengthy time periods, the stock market has generally produced positive returns.'), (3, 'During a bear market, market sentiment is negative, and investors begin to shift their money away from stocks and into fixed-income instruments as they wait for the stock market to recover. To summarise, the drop in s

### Indexing on text fields

Full-text searches are slower in larger documents. To imporve the perfomance on full-text search postgres offers option of indexing ts_vector filed which leads to faster execution of search.GIN is the most commonly used index for full-text searches. The Postgres documentation suggests two approaches:
- one is to create the to_tsvector() field directly in an index.
-  other is to first create another tsvector column in your table using to_tsvector() and then create a gin index on this field.


Lets see how second approach works

In [26]:
sql = """alter table document_text_values add column tsvector_content_desc tsvector GENERATED ALWAYS AS (to_tsvector('english', coalesce(content, '') || ' ' || coalesce(description, ''))) STORED;"""
cur = conn.cursor()
cur.execute(sql)

## Now lets create GIN index
sql = "CREATE INDEX textsearch_idx ON document_text_values USING GIN (tsvector_content_desc);"
cur = conn.cursor()
cur.execute(sql)


Note: 
- coalesce means if the column value is null, it return the defined value.Here if content is NULL it will return '' as value. 
- '||' is used for string concatenation.

In [31]:
sql = "select * from document_text_values"
execute_statement(sql)

[(1,
  'The stock exchange predicts a bull market which would make many investors happy, but analysts warn of possibility of too much optimism and that in fact we are awaiting a bear market. As always predicting the future market is an uncertain game and all investors should follow their instincts and best practices.',
  'Impact of bull market on investors',
  "'alway':34 'analyst':15 'await':29 'bear':31 'best':51 'bull':6,55 'exchang':3 'fact':26 'follow':47 'futur':37 'game':42 'happi':13 'impact':53 'instinct':49 'investor':12,45,58 'make':10 'mani':11 'market':7,32,38,56 'much':21 'optim':22 'possibl':18 'practic':52 'predict':4,35 'stock':2 'uncertain':41 'warn':16 'would':9"),
 (2,
  'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," the vast majority of investors are "bullish." Over lengthy time periods, the s

Once we have created index, we no longer need to convert text to ts_vector each time.We can directly use that column.

In [42]:
sql = "select file_id, content from document_text_values where tsvector_content_desc @@ to_tsquery('(!bear & predict)| instruments');"

execute_statement(sql)

[(3,
  'During a bear market, market sentiment is negative, and investors begin to shift their money away from stocks and into fixed-income instruments as they wait for the stock market to recover. To summarise, the drop in stock market values has shaken investor confidence. As a result, investors keep their money out of the market, causing a general price decrease as outflow grows.')]

### Ranking the Text

Ranking our search results is very important.Ranking attempts to measure how relevant documents are to a particular query, so that when there are many matches the most relevant ones can be shown first.Postgres has two predefined ranking functions, which take into account lexical, proximity, and structural information.

* ts_rank: It ranks vectors based on the frequency of their matching lexemes

* ts_rank_cd: This function computes the cover density ranking for the given document vector and query, as described in Clarke, Cormack, and Tudhope's "Relevance Ranking for One to Three Term Queries" in the journal "Information Processing and Management", 1999. Cover density is similar to ts_rank ranking except that the proximity of matching lexemes to each other is taken into consideration.



Lets see simple query for basic ranking

In [107]:


sql = "select ts_rank(to_tsvector('english','apple grow in field and is healthy'),to_tsquery('apple'))"
score = execute_statement(sql)
print(score[0][0])


sql = "select ts_rank_cd(to_tsvector('english','apple grow in field and is healthy'),to_tsquery('apple'))"
score = execute_statement(sql)
print(score[0][0])

0.06079271
0.1


In [48]:
sql_statement = "select file_id, content, tsvector_content_desc, ts_rank(tsvector_content_desc, query) AS rank FROM document_text_values, websearch_to_tsquery('bull') as query where query @@ tsvector_content_desc ORDER BY rank DESC LIMIT 3;"
execute_statement(sql_statement)

[(1,
  'The stock exchange predicts a bull market which would make many investors happy, but analysts warn of possibility of too much optimism and that in fact we are awaiting a bear market. As always predicting the future market is an uncertain game and all investors should follow their instincts and best practices.',
  "'alway':34 'analyst':15 'await':29 'bear':31 'best':51 'bull':6,55 'exchang':3 'fact':26 'follow':47 'futur':37 'game':42 'happi':13 'impact':53 'instinct':49 'investor':12,45,58 'make':10 'mani':11 'market':7,32,38,56 'much':21 'optim':22 'possibl':18 'practic':52 'predict':4,35 'stock':2 'uncertain':41 'warn':16 'would':9",
  0.075990885),
 (2,
  'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," the vast majority of investors are "bullish." Over lengthy time periods, the stock market has generally

In [115]:
sql_statement = "select file_id, content, tsvector_content_desc, ts_rank_cd(tsvector_content_desc, query) AS rank FROM document_text_values, to_tsquery('bull|bullish') as query where query @@ tsvector_content_desc ORDER BY rank DESC LIMIT 3;"
execute_statement(sql_statement)

[(1,
  'The stock exchange predicts a bull market which would make many investors happy, but analysts warn of possibility of too much optimism and that in fact we are awaiting a bear market. As always predicting the future market is an uncertain game and all investors should follow their instincts and best practices.',
  "'alway':34 'analyst':15 'await':29 'bear':31 'best':51 'bull':6,55 'exchang':3 'fact':26 'follow':47 'futur':37 'game':42 'happi':13 'impact':53 'instinct':49 'investor':12,45,58 'make':10 'mani':11 'market':7,32,38,56 'much':21 'optim':22 'possibl':18 'practic':52 'predict':4,35 'stock':2 'uncertain':41 'warn':16 'would':9",
  0.2),
 (2,
  'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," the vast majority of investors are "bullish." Over lengthy time periods, the stock market has generally produce

Here we will pass our ts_query and our search column(which is ts vectorized) to the ranking function to get the rank

**Normalization in ts_rank_cd()**

Since a longer document has a greater chance of containing a query term it is reasonable to take into account document size, e.g., a hundred-word document with five instances of a search word is probably more relevant than a thousand-word document with five instances. Both ranking functions take an integer normalization option that specifies whether and how a document's length should impact its rank. The integer option controls several behaviors, so it is a bit mask: you can specify one or more behaviors using | (for example, 2|4).

```
0 (the default) ignores the document length

1 divides the rank by 1 + the logarithm of the document length

2 divides the rank by the document length

4 divides the rank by the mean harmonic distance between extents (this is implemented only by ts_rank_cd)

8 divides the rank by the number of unique words in document

16 divides the rank by 1 + the logarithm of the number of unique words in document

32 divides the rank by itself + 1
```

If more than one flag bit is specified, the transformations are applied in the order listed.


In [121]:
sql_statement = """SELECT file_id, content, ts_rank_cd(tsvector_content_desc,searchvector, 32) AS rank FROM document_text_values, plainto_tsquery('bull market') AS searchvector WHERE searchvector @@ tsvector_content_desc ORDER BY rank DESC"""



execute_statement(sql_statement)

[(1,
  'The stock exchange predicts a bull market which would make many investors happy, but analysts warn of possibility of too much optimism and that in fact we are awaiting a bear market. As always predicting the future market is an uncertain game and all investors should follow their instincts and best practices.',
  0.17073171),
 (4,
  ' the decrease in stock market values has shaken investor confidence, As they would have predicted that the market will fall more.  This causes investors to keep their money out of the market, causing a general price decrease as outflow grows. In bear market, value of stocks falls while in bull market the value of stocks falls. However the market has been producing the positive result over length periods.',
  0.102564104),
 (2,
  'A bull market is one in which the market is rising and the economy is doing well, whereas a bear market is one in which the economy is collapsing and most equities are losing value. Although some investors are "bearish," t